<a href="https://colab.research.google.com/github/ameya932/web_scraping/blob/main/Amazon_Web_Scraping_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()
    except AttributeError:
        available = "Not Available"	

    return available

In [4]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=snacks&crid=VT116UMCKZMJ&sprefix=snacks%2Caps%2C438&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [5]:
amazon_df

,title,price,rating,reviews,availability
0,Apples A Day - Kudu Kudu - Multi Millet Popped...,,5.0 out of 5 stars,5 ratings,In stock
1,The Healthy Binge Ragi Chips Baked Chips Snack...,,4.2 out of 5 stars,196 ratings,In stock
2,Orion O'rice cracker - Baked Korean snack (Pac...,,4.3 out of 5 stars,512 ratings,In stock
3,Orion Turtle Chips Assorted snacks box - 5 fla...,,5.0 out of 5 stars,2 ratings,In stock
4,Beyond Snáck Kerala Banana Chips | 3 Pack Comb...,,3.8 out of 5 stars,"3,373 ratings",In stock
5,Chheda's - Yellow Banana Chips - Banana Wafers...,,4.3 out of 5 stars,"1,076 ratings",In stock
6,Farmley Roasted & Flavored Makhana- Minty Pudi...,,4.1 out of 5 stars,931 ratings,In stock
7,Orion O'rice cracker - Baked Korean snack (Pac...,,4.3 out of 5 stars,512 ratings,In stock
8,"UNIBIC Cookies, Assorted Cookies, 75 g (Pack o...",,4.2 out of 5 stars,"10,687 ratings",In stock
9,Beyond Snáck Kerala Banana Chips - Pack of 4 C...,,3.8 out of 5 stars,460 ratings,In stock
